In [1]:
import tensorflow as tf

/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/ten

# 模拟先同步处理数据，然后取数据训练

In [3]:
# 1.首先定义队列
Q = tf.FIFOQueue(3,tf.float32)

#2.放入数据,tensorflow默认把 []等当做张量,修改如同 (1)和(1,)的区别
enq_many = Q.enqueue_many([ [0.1,0.2,0.3], ])

# 3.定义取数据过程
out_q = Q.dequeue()
data = out_q + 1
en_q = Q.enqueue(data)

with tf.Session() as sess:
    #初始化队列
    sess.run(enq_many)
    
    #处理数据
    for i in range(100):
        sess.run(en_q)
        
    #训练数据
    for i in range(Q.size().eval()):
        sum = sess.run(Q.dequeue())
        print(sum)

33.2
33.3
34.1


# 模拟异步子线程：存入样本，主线程：读取样本

In [5]:
# 1.定义一个队列，1000
Q = tf.FIFOQueue(1000,tf.float32)
# 2.定义子线程需要处理的事：循环+1，放入队列
var = tf.Variable(0.0)
# 实现一个自增 tf.assign_add
data = tf.assign_add(var,tf.constant(1.0))
en_q = Q.enqueue(data)
# 3.定义队列管理器op，指定多少个子线程需要处理的事情
qr = tf.train.QueueRunner(Q,enqueue_ops=[en_q]*2)

#初始化变量op
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    #初始化变量
    sess.run(init_op)
    
    #开启线程管理器
    coord = tf.train.Coordinator()
    
    #真正开启子线程
    threads = qr.create_threads(sess,coord=coord,start=True)
    
    #主线程，不断读取数据训练
    for i in range(300):
        print(sess.run(Q.dequeue() ))
        
    #主线程结束，意味着session关闭，意味着资源释放
    #主线程结束前询问
    coord.request_stop()
    coord.join(threads)

2.0
10.0
25.0
42.0
61.0
79.0
100.0
118.0
140.0
159.0
175.0
195.0
214.0
240.0
256.0
278.0
300.0
314.0
329.0
346.0
367.0
391.0
412.0
429.0
449.0
468.0
488.0
514.0
535.0
558.0
579.0
661.0
681.0
699.0
717.0
740.0
758.0
782.0
810.0
829.0
851.0
866.0
883.0
911.0
930.0
948.0
963.0
981.0
1004.0
1029.0
1049.0
1050.0
1050.0
1051.0
1053.0
1053.0
1055.0
1055.0
1056.0
1057.0
1058.0
1059.0
1060.0
1061.0
1062.0
1064.0
1064.0
1065.0
1067.0
1067.0
1069.0
1069.0
1070.0
1072.0
1072.0
1073.0
1074.0
1075.0
1077.0
1078.0
1079.0
1079.0
1080.0
1081.0
1083.0
1084.0
1084.0
1086.0
1086.0
1087.0
1088.0
1089.0
1090.0
1092.0
1092.0
1093.0
1095.0
1095.0
1096.0
1098.0
1098.0
1100.0
1101.0
1101.0
1103.0
1103.0
1105.0
1105.0
1106.0
1107.0
1108.0
1109.0
1110.0
1111.0
1112.0
1113.0
1114.0
1116.0
1116.0
1117.0
1118.0
1119.0
1120.0
1121.0
1122.0
1123.0
1125.0
1125.0
1126.0
1127.0
1129.0
1129.0
1130.0
1131.0
1133.0
1134.0
1135.0
1135.0
1137.0
1137.0
1138.0
1140.0
1140.0
1142.0
1142.0
1143.0
1144.0
1146.0
1146.0
1148.0
1148.

Exception in thread QueueRunnerThread-fifo_queue_3-fifo_queue_3_enqueue:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1279, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/home/tang/mnt/F/学习/人工智能/venv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1429, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_3_enqueue}}]]

Exception in thread QueueRunnerThread-fifo_queue_3-fifo_queu